---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **DatetimeOutputParser**

* `DatetimeOutputParser` = `LLM`의 출력을 `datetime 형식`으로 파싱하는 데 사용

In [1]:
# 1_새 프롬프트 생성하기
from langsmith import Client
from langchain.prompts import PromptTemplate                            # Langchain에서 프롬프트 템플릿을 만들기 위한 모듈 임포트
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json


# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [2]:
from langchain.output_parsers import DatetimeOutputParser               # 데이트타임 형식 파싱을 위한 임포트

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langsmith import traceable                                     # LangSmith 추적 설정

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

---

* `datetime` 형식 참고

  * ![datetime 형식 참고](../data/datetimeparser_reference.png)

In [ ]:
# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt_datetime = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },                                                          # 지침을 템플릿에 적용
)

# 프롬프트 내용을 출력
prompt_datetime


<small>

* 셀 출력
  
    ```python
    PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 2025-08-25, 2024-08-25, 2025-08-24\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')
    ```

In [7]:
# 체인 생성
chain_datetime = prompt_datetime | gemini_lc | output_parser

# 체인을 호출하여 질문에 대한 답변받기
output = chain_datetime.invoke({"question": "Google 이 창업한 연도는?"})

In [ ]:
# 결과 출력하기 = 문자열로 변환하여 출력

output.strftime("%Y-%m-%d")                         # '1998-09-04' (0.0s)

In [ ]:
# 다른 질문 해보기
output2 = chain_datetime.invoke({"question": "오늘 날짜를 시스템 시간에 맞게 출력해주세요."})

# 문자열로 변환하여 출력
print(output2.strftime("%Y-%m-%d"))                         # 2024-05-15 (0.8s)

In [ ]:
# 시스템 시간을 코드에서 가져와 프롬프트에 전달하기
import datetime

current_date = datetime.datetime.now().strftime("%Y-5m-%d")

# 프롬프트로 만들기
df_datetime = f"오늘 날짜는 {current_date}입니다."

# 체인 실행하기
output3 = chain_datetime.invoke({"question":df_datetime})

# 결과 출력해보기 '
print(output3.strftime("%Y-%m-%d"))                         # 2025-05-25 (0.7s)

<small>

* 문제 발생 - **현재 날짜 출력** 묻는 프롬프트에 무조건 **`2024-05-15`** 으로 답하는 현상
  
  * `output2`의 첫 프롬프트: `대한민국의 현재 날짜는?`
    * 구체적인 장소를 제공하여 현재 날짜를 출력할 것으로 기대
    * `but` → 출력값: `2024-05-15`
  
  * `두번째 시도`: `현재 날짜는?` or `오늘 날짜는?`
    * `현재`, `오늘` 이라는 단어를 통해 현재 날짜를 알 수 있을 것으로 기대
    * `but` → 출력값: `2024-05-15`
  
  * `세번째 시도`: `LLM`이 학습된 데이터의 일부가 `특정 기준 날짜를 가지고 있을 수 있을 가능성`
    * 즉, **`2024-05-25`가 기본 설정값** 으로 설정된 경우일 가능성
    * 기본값을 무시하도록 프롬프트 수정 
      * `기본 날짜 설정을 무시하고, 오늘 날짜를 알려주세요.`
      * `오늘 날짜를 시스템 시간에 맞게 출력해주세요.`
    * **여전히 `2024-05-15`로 출력**
  
  * `gemini-2.5-flash-lite` = **모델 자체에 실시간 시스템 시간에 접근할 수 있는 제한이 없을 것으로 추측**
    * 추측 이유
      * 이유_1: **모델의 시간 처리 방식**
        * `Gemini-2.5-Flash-light` 모델 = 학습된 데이터에 기반하여 응답을 생성
        * 실시간 날짜 정보를 사용할 수 없거나 시스템 시간에 접근할 수 없을 수 있음
  
      * 이유_2: **프롬프트 자체에 시스템 시간 요청이 불가능**
        * `LLM`이 **훈련된 데이터와 규칙을 바탕으로 작동**
        * 따라서 실시간 시스템 시간을 모델에 직접 반영하는 것은 일반적인 사용 방식에서는 불가능할 수 있음
  
    * 해결방법
      * 해결방법_1: `Python 코드`를 사용하여 시스템 날짜를 가져오기
        * `Python 코드`로 `시스템 날짜`를 동적으로 가져오고, 그 값을 프롬프트에 넣어서 모델을 호출하기
        * **성공!**

      * 해결방법_2: 실시간 시간에 접근 가능한 `LLM`모델 사용하기
        * `ChatGPT`의 **플러그인 및 `API` 기능을 통해 시간 관련 작업 처리 가능**
        * `Google PaLM` 
          * `Google Cloud의 AI API` 
          * `PaLM` 모델 = 실시간 API 호출, 외부 데이터 소스에 통합해 시스템 시간 사용하는 APP 구축 가능
        * `Claude`: 시간, 날짜에 대한 문제를 해결할 수 있는 API 제공

---

* *next: EnumOutputParser(열거형 출력 파서)*